In [ ]:
using NamedColors
using Colors
using ColorTypes
using StatsBase

In [ ]:
training_dict = load_xkcd();

In [ ]:
x_raw_1="blood"
y_raw_1 = training_dict[x_raw_1]

In [ ]:
function encode_output(raw::Colorant)
    #Encoding Angles: http://stats.stackexchange.com/a/218547/36769
    hsv = convert(HSV, raw)
    [(sind(hsv.h)+1)/2, (cosd(hsv.h)+1)/2, hsv.s, hsv.v]
end

function decode_output(out)
    hue = rad2deg(atan2(2out[1]-1, 2out[2]-1))
    sat = out[3]
    val = out[4]
    HSV(hue, sat, val)
end

y1 = encode_output(y_raw_1)
@show y1
decode_output(y1)

In [ ]:
function char2index(c::Char)::Int
    if c ∈ 'a':'z'
        c-'a'+1
    elseif c==' '
        'z'+1+1
    elseif c=='/'
        'z'+2+1
    elseif c=='\''
        'z'+3+1
    else
        error("$c is out of range")
    end
end
function encode_input(name::AbstractString)
    name_inds::Vector{Int} = char2index.(collect(name))
    counts(name_inds, 26+3)./length(name)
end

In [ ]:
using Learn
import MNIST
using MLDataUtils
using StatsBase
using StatPlots
using Plots

# Set up GR for plotting. x11 is uglier, but much faster
#ENV["GKS_WSTYPE"] = "x11"
gr(leg=false, linealpha=0.5)


In [ ]:
function get_training(training_dict)

    colors = collect(values(training_dict))
    y_train = hcat(encode_output.(colors)...)
    
    names = collect(keys(training_dict))
    x_train = hcat(encode_input.(names)...)
    x_train, y_train
end


In [ ]:
train = get_training(training_dict)


In [ ]:
function my_test_loss(obj, testdata, totcount = 500)
    totloss = 0.0
    totcorrect = 0
    for (x,y) in eachobs(rand(eachobs(testdata), totcount))
        totloss += transform!(obj,y,x)
        #ŷ = output_value(obj.transformation)[1]
        #correct = (ŷ > 0.5 && y > 0.5) || (ŷ <= 0.5 && y < 0.5)
        #totcorrect += correct
    end
    totloss#, totcorrect/totcount
end


tracer = IterFunction((obj, i) -> begin
    # compute approximate test loss and trace it
    if mod1(i,1000)==1000
        totloss  = my_test_loss(obj, train, 500)
        println(i, " totloss:  ", totloss)
        #println(i, " accuracy: ", accuracy)
    end

end)

# trace once before we start learning to see initial values
tracer.f(obj, 0)



In [ ]:
nin, nh, nout = size(train[1],1), [100,4^3], size(train[2],1)
t = nnet(nin, nout, nh, :softplus, :logistic)
obj = objective(t, ElasticNetPenalty(1e-5))

In [ ]:
learner = make_learner(
    GradientLearner(5e-2, Adadelta()),
    tracer,
    maxiter = 500_000
)
learn!(obj, learner, infinite_batches(train, size=5))

In [ ]:
# the type, parameterized by the indices and plotting backend
type TracePlot{I,T}
    indices::I
    plt::Plot{T}
end

getplt(tp::TracePlot) = tp.plt

# construct a TracePlot for n series.  note we pass through
# any keyword arguments to the `plot` call
function TracePlot(n::Int = 1; maxn::Int = 500, kw...)
    indices = if n > maxn
        # limit to maxn series, randomly sampled
        shuffle(1:n)[1:maxn]
    else
        1:n
    end
    TracePlot(indices, plot(length(indices); kw...))
end

# add a y-vector for value x
function add_data(tp::TracePlot, x::Number, y::AbstractVector)
    for (i,idx) in enumerate(tp.indices)
        push!(tp.plt.series_list[i], x, y[idx])
    end
end

# convenience: if y is a number, wrap it as a vector and call the other method
add_data(tp::TracePlot, x::Number, y::Number) = add_data(tp, x, [y])

In [ ]:
# our data:
x_train, y_train = MNIST.traindata()
x_test, y_test = MNIST.testdata()

# normalize the input data given μ/σ for the input training data
μ, σ = rescale!(x_train)
rescale!(x_test, μ, σ)

# convert class vector to "one hot" matrix
y_train, y_test = map(to_one_hot, (y_train, y_test))

train = (x_train, y_train)
test = (x_test, y_test)

In [ ]:
nin, nh, nout = 784, [50,50], 10
t = nnet(nin, nout, nh, :softplus, :softmax)

In [ ]:
x_train |> size

In [ ]:
obj = objective(t, ElasticNetPenalty(1e-5))

In [ ]:
# parameter plots
pidx = 1:2:length(t)
pvalplts = [TracePlot(length(params(t[i])), title="$(t[i])") for i=pidx]
ylabel!(pvalplts[1].plt, "Param Vals")
pgradplts = [TracePlot(length(params(t[i]))) for i=pidx]
ylabel!(pgradplts[1].plt, "Param Grads")

# nnet plots of values and gradients
valinplts = [TracePlot(input_length(t[i]), title="input", yguide="Layer Value") for i=1:1]
valoutplts = [TracePlot(output_length(t[i]), title="$(t[i])", titlepos=:left) for i=1:length(t)]
gradinplts = [TracePlot(input_length(t[i]), yguide="Layer Grad") for i=1:1]
gradoutplts = [TracePlot(output_length(t[i])) for i=1:length(t)]

# loss/accuracy plots
lossplt = TracePlot(title="Test Loss", ylim=(0,Inf))
accuracyplt = TracePlot(title="Accuracy", ylim=(0.6,1))

In [ ]:
function my_test_loss(obj, testdata, totcount = 500)
    totloss = 0.0
    totcorrect = 0
    for (x,y) in eachobs(rand(eachobs(testdata), totcount))
        totloss += transform!(obj,y,x)

        # logistic version:
        # ŷ = output_value(obj.transformation)[1]
        # correct = (ŷ > 0.5 && y > 0.5) || (ŷ <= 0.5 && y < 0.5)

        # softmax version:
        ŷ = output_value(obj.transformation)
        chosen_idx = indmax(ŷ)
        correct = y[chosen_idx] > 0

        totcorrect += correct
    end
    totloss, totcorrect/totcount
end

In [ ]:
tracer = IterFunction((obj, i) -> begin
    # compute approximate test loss and trace it
    if mod1(i,500)==500
        totloss, accuracy = my_test_loss(obj, test, 500)
        println(i, " totloss:  ", totloss)
        println(i, " accuracy: ", accuracy)
    end

end)

# trace once before we start learning to see initial values
tracer.f(obj, 0)



In [ ]:
learner = make_learner(
    GradientLearner(5e-2, Adadelta()),
    tracer,
    maxiter = 50000
)
learn!(obj, learner, infinite_batches(train, size=5))



In [ ]:
function get_colour(obj, name)
    transform!(obj,[0,0,0,0f0],encode_input(name))
    decode_output( output_value(obj.transformation ))
end
function display_colour(obj,name)
    display(name)
    display(get_colour(obj, name))
    display("-"^16)
end

In [124]:
display_colour(obj, "green")
display_colour(obj, "red")
display_colour(obj, "lightred")
display_colour(obj, "dirtyred")
display_colour(obj, "lightredishblue")
display_colour(obj, "bluish red")
display_colour(obj, "blue")
display_colour(obj, "light blue")
display_colour(obj, "lightred")



"green"

"----------------"

"red"

"----------------"

"lightred"

...

"----------------"

"light blue"

"----------------"

"lightred"

"----------------"

In [131]:
display_colour(obj, "hot pink")
display_colour(obj, "hot blue")
display_colour(obj, "hot green")
display_colour(obj, "dark blue")
display_colour(obj, "light black")
display_colour(obj, "black")
display_colour(obj, "dark black")

"hot pink"

"----------------"

"hot blue"

"----------------"

"hot green"

...

"----------------"

"black"

"----------------"

"dark black"

"----------------"

In [135]:
for letter in ['a':'z'; ' '; '\''; '/']
    display_colour(obj, string(letter))
end

"a"

"----------------"

"b"

"----------------"

"c"

...

"----------------"

"'"

"----------------"

"/"

"----------------"

In [136]:
for letter1 in ['a':'z'; ' '; '\''; '/']
    for letter2 in ['a':'z'; ' '; '\''; '/']
        display_colour(obj, join([letter1,letter2]))
    end
end

"aa"

"----------------"

"ab"

"----------------"

"ac"

...

"----------------"

"/'"

"----------------"

"//"

"----------------"